<a href="https://colab.research.google.com/github/sammitako/CS-GY-6513-Big-Data/blob/master/Keyword_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
!pip install taxonerd

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [14]:
!pip install --upgrade spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
scispacy 0.5.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.7.2 which is incompatible.
taxonerd 1.5.2 requires spacy<3.5.0,>3.4.0, but you have spacy 3.7.2 which is incompatible.


In [2]:
# import libraries
import spacy
import csv
import pandas as pd
import numpy as np
import re
from collections import Counter
from taxonerd import TaxoNERD

# Data pre-processing on 'product_fingerprint' column
> Upon analyzing the data, we discovered that the 'name', 'title', and 'product' columns contain identical data in the field. Therefore, we chose the 'product' column as it has the longest average text length.


To extract the species from each row, we tokenize the product_fingerprint column in an attempt to find relevant keywords. But the column contains a lot of text, most of which are irrelevant to the species name. By removing these words, we remove the low-level information from the product description in order to give more focus to the species name.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Big Data/Team Notebook/data/preprocessed_data_file_geography_price_removed_outliers.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51610 entries, 0 to 51609
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     51610 non-null  object 
 1   title                    51610 non-null  object 
 2   product                  51610 non-null  object 
 3   description              51328 non-null  object 
 4   domain                   51610 non-null  object 
 5   country                  41690 non-null  object 
 6   label_product            51610 non-null  object 
 7   price                    51610 non-null  float64
 8   currency                 51610 non-null  object 
 9   production_date          14 non-null     object 
 10  image                    51307 non-null  object 
 11  url                      51610 non-null  object 
 12  location                 5353 non-null   object 
 13  domain_country_code      51610 non-null  object 
 14  domain_country        

In [4]:
avg_length_product = df['product_fingerprint'].apply(lambda x: len(str(x))).mean()
avg_length_name = df['name_fingerprint'].apply(lambda x: len(str(x))).mean()
avg_length_title = df['title_fingerprint'].apply(lambda x: len(str(x))).mean()

avg_lengths = {
    'product_fingerprint': avg_length_product,
    'name_fingerprint': avg_length_name,
    'title_fingerprint': avg_length_title
}

# Find the column with the longest average text length
column_with_longest_avg = max(avg_lengths, key=avg_lengths.get)
print(f"The column with the longest average text length is: {column_with_longest_avg} with an average length of {avg_lengths[column_with_longest_avg]}")

The column with the longest average text length is: product_fingerprint with an average length of 65.43516760317767


Drop the columns 'name', 'title', 'product', and 'description' from DataFrame and then rename the `*_fingerprint` columns to just the base column names (like 'name')

In [5]:
df = df.drop(columns=['name', 'title', 'product', 'description'])

df = df.rename(columns={
    'name_fingerprint': 'name',
    'title_fingerprint': 'title',
    'product_fingerprint': 'product',
    'description_fingerprint': 'description'
})

# Reordering the columns
columns_first = ['name', 'title', 'product', 'description']
remaining_columns = [col for col in df.columns if col not in columns_first]
new_column_order = columns_first + remaining_columns
df = df[new_column_order]
df.head()

,name,title,product,description,domain,country,label_product,price,currency,production_date,image,url,location,domain_country_code,domain_country,normalized_price_usd
0,adventures and bones color ebay hailstones hal...,adventures and bones color ebay hailstones hal...,adventures and bones color ebay hailstones hal...,adventures and at best bones color deals ebay ...,ebay.com.au,Australia,an animal body part,22.25,AUD,NaN,https://images.awesomebooks.com/images/books/r...,https://www.ebay.com.au/itm/384104329587?amdat...,NaN,au,Australia,34
1,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals delivery ebay find...,ebay.co.uk,United Kingdom,an animal body part,12.63,GBP,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.co.uk/itm/222613331802,NaN,uk,United Kingdom,10
2,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for get ...,ebay.com.au,Australia,an animal body part,23.93,AUD,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.com.au/itm/222613331802,NaN,au,Australia,36
3,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,collectibles contortus ebay fossils hill miner...,ebay.nl,Netherlands,an animal body part,14.40,EUR,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.nl/itm/222613331802,NaN,nl,Netherlands,13
4,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for free...,ebay.ca,Canada,an animal body part,20.30,CAD,NaN,https://i.ebayimg.com/images/g/ORAAAOSwk-pZkbC...,https://www.ebay.ca/itm/222613236770,NaN,ca,Canada,28


## Removing duplicates

In [6]:
# product_fingerprint column.
df['product']

0        adventures and bones color ebay hailstones hal...
1        contortus ebay hill miocene physogaleus shark ...
2        contortus ebay hill miocene physogaleus shark ...
3        contortus ebay hill miocene physogaleus shark ...
4        contortus ebay hill miocene physogaleus shark ...
                               ...                        
51605     bone ebay fl iiblue megalodon shark tooth valley
51606     bone ebay fl iiblue megalodon shark tooth valley
51607    ebay funnyauiiii iiiigreat large shark size ts...
51608    ebay funnyauiiii iiiigreat large shark size ts...
51609    ebay funnyauiiii iiiigreat large shark size ts...
Name: product, Length: 51610, dtype: object

In [7]:
# The unique values that have duplicates
duplicate_values = df.loc[df['product'].duplicated(keep='first'), 'product'].unique()

print(len(duplicate_values))

6544


In [57]:
print(duplicate_values[0])

contortus ebay hill miocene physogaleus shark sharktooth tiger tooth


In [8]:
first_duplicate_value = duplicate_values[0]
rows_with_first_duplicate = df[df['product'] == first_duplicate_value]

rows_with_first_duplicate

,name,title,product,description,domain,country,label_product,price,currency,production_date,image,url,location,domain_country_code,domain_country,normalized_price_usd
1,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals delivery ebay find...,ebay.co.uk,United Kingdom,an animal body part,12.63,GBP,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.co.uk/itm/222613331802,NaN,uk,United Kingdom,10
2,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for get ...,ebay.com.au,Australia,an animal body part,23.93,AUD,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.com.au/itm/222613331802,NaN,au,Australia,36
3,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,collectibles contortus ebay fossils hill miner...,ebay.nl,Netherlands,an animal body part,14.40,EUR,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.nl/itm/222613331802,NaN,nl,Netherlands,13
4,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for free...,ebay.ca,Canada,an animal body part,20.30,CAD,NaN,https://i.ebayimg.com/images/g/ORAAAOSwk-pZkbC...,https://www.ebay.ca/itm/222613236770,NaN,ca,Canada,28
5,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for get ...,ebay.com.au,Australia,an animal body part,23.48,AUD,NaN,https://i.ebayimg.com/images/g/ORAAAOSwk-pZkbC...,https://www.ebay.com.au/itm/222613236770,NaN,au,Australia,36
6,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for free...,ebay.com,NaN,an animal body part,14.80,USD,NaN,https://i.ebayimg.com/images/g/ORAAAOSwk-pZkbC...,https://www.ebay.com/itm/222613236770,NaN,com,NaN,15
7,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for free...,ebay.com,NaN,an animal body part,115.89,HKD,NaN,https://i.ebayimg.com/images/g/ORAAAOSwk-pZkbC...,https://www.ebay.com/itm/222613236770?_ul=HK,NaN,com,NaN,903
8,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,collectibles contortus ebay fossils hill miner...,ebay.nl,Netherlands,an animal body part,14.10,EUR,NaN,https://i.ebayimg.com/images/g/ORAAAOSwk-pZkbC...,https://www.ebay.nl/itm/222613236770,NaN,nl,Netherlands,13
9,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals delivery ebay find...,ebay.co.uk,United Kingdom,an animal body part,10.56,GBP,NaN,https://i.ebayimg.com/images/g/-NAAAOSwweRZkxO...,https://www.ebay.co.uk/itm/222614233860,NaN,uk,United Kingdom,8
10,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals ebay find for free...,ebay.com,NaN,an animal body part,12.80,USD,NaN,https://i.ebayimg.com/images/g/-NAAAOSwweRZkxO...,https://www.ebay.com/itm/222614233860,NaN,com,NaN,13


We observed that the same advertisements are posted on different URLs. Upon examination, we noted that the 'name', 'title', and 'product' are the same; however, the descriptions vary:

- Remove duplicate rows where 'name', 'title', and 'product' are the same.
- Remove duplicate rows where the description is the same.

In [9]:
# Step 1
df = df.drop_duplicates(subset=['name', 'title', 'product'])

# Step 2
df = df.drop_duplicates(subset=['description'])

In [10]:
# name, title, product: 51610 -> 17433 (-34177)
# description: 51306 -> 17432 (-33874)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17433 entries, 0 to 51607
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  17433 non-null  object 
 1   title                 17433 non-null  object 
 2   product               17433 non-null  object 
 3   description           17432 non-null  object 
 4   domain                17433 non-null  object 
 5   country               14209 non-null  object 
 6   label_product         17433 non-null  object 
 7   price                 17433 non-null  float64
 8   currency              17433 non-null  object 
 9   production_date       14 non-null     object 
 10  image                 17412 non-null  object 
 11  url                   17433 non-null  object 
 12  location              4744 non-null   object 
 13  domain_country_code   17433 non-null  object 
 14  domain_country        9544 non-null   object 
 15  normalized_price_us

In [61]:
# product_fingerprint column.
df.head(10)

,name,title,product,description,domain,country,label_product,price,currency,production_date,image,url,location,domain_country_code,domain_country,normalized_price_usd
0,adventures and bones color ebay hailstones hal...,adventures and bones color ebay hailstones hal...,adventures and bones color ebay hailstones hal...,adventures and at best bones color deals ebay ...,ebay.com.au,Australia,an animal body part,22.25,AUD,NaN,https://images.awesomebooks.com/images/books/r...,https://www.ebay.com.au/itm/384104329587?amdat...,NaN,au,Australia,34
1,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,contortus ebay hill miocene physogaleus shark ...,and at best contortus deals delivery ebay find...,ebay.co.uk,United Kingdom,an animal body part,12.63,GBP,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.co.uk/itm/222613331802,NaN,uk,United Kingdom,10
14,azaz contortus ebay hill physogaleus sharktooth,azaz contortus ebay hill physogaleus sharktooth,azaz contortus ebay hill physogaleus sharktooth,and at azaz best contortus deals ebay find for...,ebay.com,NaN,an animal body part,111.39,CNY,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.com/itm/222613331802?_ul=CN,NaN,com,NaN,790
15,contortus de dent ebay hill miocune physogaleu...,contortus de dent ebay hill miocune physogaleu...,contortus de dent ebay hill miocune physogaleu...,comparez contortus darticles de dent des docca...,ebay.fr,France,an animal body part,12.22,EUR,NaN,https://i.ebayimg.com/images/g/~M8AAOSwjuBZlzv...,https://www.ebay.fr/itm/222617333943?amdata=en...,NaN,fr,France,11
16,colline contortus de dent ebay miocune physoga...,colline contortus de dent ebay miocune physoga...,colline contortus de dent ebay miocune physoga...,colline comparez contortus darticles de dent d...,ebay.fr,France,an animal body part,14.53,EUR,NaN,https://i.ebayimg.com/images/g/x~gAAOSwtsVZkch...,https://www.ebay.fr/itm/222613331802,NaN,fr,France,13
33,almofada brinquedo d de ebay estereo gato peca...,almofada brinquedo d de ebay estereo gato peca...,almofada brinquedo d de ebay estereo gato peca...,almofada and at best brinquedo d de deals ebay...,ebay.com,NaN,an animal body part,74.43,EUR,NaN,https://i.ebayimg.com/images/g/CQkAAOSwbtFlGxO...,https://www.ebay.com/itm/155807567987?_ul=PT,NaN,com,NaN,68
34,cuscino d ebay gatto giocattolo imbottito pelu...,cuscino d ebay gatto giocattolo imbottito pelu...,cuscino d ebay gatto giocattolo imbottito pelu...,articoli caratteristiche con confronta consegn...,ebay.it,Italy,an animal body part,74.43,EUR,NaN,https://i.ebayimg.com/images/g/CQkAAOSwbtFlGxO...,https://www.ebay.it/itm/155807567987?amdata=en...,NaN,it,Italy,68
37,black crafts decorations dining ebay feathers ...,black crafts decorations dining ebay feathers ...,black crafts decorations dining ebay feathers ...,and at best black crafts deals decorations del...,ebay.co.uk,United Kingdom,an animal body part,9.58,GBP,NaN,https://i.ebayimg.com/images/g/Q1IAAOSws29lGwl...,https://www.ebay.co.uk/itm/314864391496?amdata...,NaN,uk,United Kingdom,8
50,black centerpiece ebay feathers large ostrich ...,black centerpiece ebay feathers large ostrich ...,black centerpiece ebay feathers large ostrich ...,and at best black centerpiece deals delivery e...,ebay.co.uk,United Kingdom,an animal body part,8.31,GBP,NaN,https://i.ebayimg.com/images/g/UukAAOSwlfJlGwl...,https://www.ebay.co.uk/itm/314864424353?amdata...,NaN,uk,United Kingdom,7
67,assorted colors craft crown ebay feathers larg...,assorted colors craft crown ebay feathers larg...,assorted colors craft crown ebay feathers larg...,and assorted at best colors craft crown deals ...,ebay.ca,Canada,an animal body part,12.79,CAD,NaN,https://i.ebayimg.com/images/g/4IQAAOSwmHtlGwk...,https://www.ebay.ca/itm/314864388058,NaN,ca,Canada,17


In [11]:
final_file_path = '/content/drive/MyDrive/Big Data/Team Notebook/data/final_cleaned_data.csv'
df.to_csv(final_file_path, index=False)

## Removing stopwords
Stopwords are the most common words in any language (ex. articles, prepositions, pronouns, conjunctions, etc) and does not add much information to the text. Examples of a few stop words in English are “the”, “a”, “an”, “so”, “what”.
The removal of such words will not have any negative consequences for our task. It reduces the number of tokens and leads to a more accurate species extraction.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Big Data/Team Notebook/data/final_cleaned_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17433 entries, 0 to 17432
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  17433 non-null  object 
 1   title                 17433 non-null  object 
 2   product               17433 non-null  object 
 3   description           17432 non-null  object 
 4   domain                17433 non-null  object 
 5   country               14209 non-null  object 
 6   label_product         17433 non-null  object 
 7   price                 17433 non-null  float64
 8   currency              17433 non-null  object 
 9   production_date       14 non-null     object 
 10  image                 17412 non-null  object 
 11  url                   17433 non-null  object 
 12  location              4744 non-null   object 
 13  domain_country_code   17433 non-null  object 
 14  domain_country        9544 non-null   object 
 15  normalized_price_us

In [4]:
# Loading the english language small model of spacy
en = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
spacy_stop_words = en.Defaults.stop_words

In [6]:
spacy_stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

In [7]:
 # Remove stop words from product
df['product'] = df['product'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in spacy_stop_words]))

# product column with stopwords removed.
df['product'].head(10)

0    adventures bones color ebay hailstones halibut...
1    contortus ebay hill miocene physogaleus shark ...
2      azaz contortus ebay hill physogaleus sharktooth
3    contortus de dent ebay hill miocune physogaleu...
4    colline contortus de dent ebay miocune physoga...
5    almofada brinquedo d de ebay estereo gato peca...
6    cuscino d ebay gatto giocattolo imbottito pelu...
7    black crafts decorations dining ebay feathers ...
8    black centerpiece ebay feathers large ostrich ...
9    assorted colors craft crown ebay feathers larg...
Name: product, dtype: object

### Remove all numerals and irrelevant common words.
Remove all numerals as it is irrelevant to species name.

In [8]:
# Remove numerals using regular expression.
df['product'] = df['product'].apply(lambda x: ' '.join([re.sub(r'\d+', '', word) for word in x.split()]))

# product column with numerals removed.
df['product'].head(10)

0    adventures bones color ebay hailstones halibut...
1    contortus ebay hill miocene physogaleus shark ...
2      azaz contortus ebay hill physogaleus sharktooth
3    contortus de dent ebay hill miocune physogaleu...
4    colline contortus de dent ebay miocune physoga...
5    almofada brinquedo d de ebay estereo gato peca...
6    cuscino d ebay gatto giocattolo imbottito pelu...
7    black crafts decorations dining ebay feathers ...
8    black centerpiece ebay feathers large ostrich ...
9    assorted colors craft crown ebay feathers larg...
Name: product, dtype: object

Find the most common words that are irrelevant for species extraction and remove them from the product fingerprint column.

In [9]:
# The most common 800 words.
[i[0] for i in Counter(" ".join(df['product']).split()).most_common()[:800]]

['ebay',
 'picclick',
 'shark',
 'wolf',
 'ostrich',
 'white',
 'great',
 'crocodile',
 'de',
 'mens',
 'canis',
 'lupus',
 'tooth',
 'new',
 'animal',
 'size',
 'black',
 'leather',
 'nile',
 'vintage',
 'tiger',
 'boots',
 'd',
 'x',
 'figure',
 'fossil',
 'fish',
 'halibut',
 'cowboy',
 'tshirt',
 'beetle',
 'head',
 'mm',
 'blue',
 'brown',
 's',
 'teeth',
 'ebid',
 'red',
 'sharks',
 'genuine',
 'shirt',
 'jaws',
 'tigre',
 'cm',
 'western',
 'alligator',
 'necklace',
 'megalodon',
 'trade',
 'mnh',
 'marketplace',
 'set',
 'pendant',
 't',
 'skin',
 'real',
 'xl',
 'sale',
 'large',
 'silver',
 'wild',
 'm',
 'oz',
 'sand',
 'united',
 'usa',
 'camelus',
 'toy',
 'carabidae',
 'feather',
 'redfish',
 'struthio',
 'rare',
 'pcs',
 'b',
 'men',
 'wolves',
 'lot',
 'kingdom',
 'plush',
 'fasciatus',
 'l',
 'decor',
 'quill',
 'bird',
 'taxidermy',
 'c',
 'tiburon',
 'bag',
 'toe',
 'niloticus',
 'inch',
 'w',
 'art',
 'insect',
 'natural',
 'model',
 'en',
 'card',
 'sea',
 'exotic'

From the most 800 common words, we make a list of words that we assume have no relevance to species name.

In [10]:
common_words = ['white','de','great','nile','tooth','sand','animal','mnh','new','mens','x','black','size','mm',
                'ground','leather','fossil','silky','vintage','teeth','d','tiburon','s','cm','genuine','sale',
                'decor','jaws','upaep','necklace','boots','blue','head','inch','plush','shirt','ss','bird','red',
                'model','baby','arena','fauna','uruguay','lot','strau','real','t','rare','brown','jaw','cowboy',
                'pendant','b','skin','en','el','c','large','stamp','set','xl','nilo','silver','del','sj','fdc','m',
                'oz','usa','l','card','mi','cm','usa','di','western','feather','taxidermy','wallet','bag','color',
                'von','coleoptera','beanie','ty','proof','specimen','florida','xl','sp','epl','wild','hide','small',
                'men','stuffed','nil','spotted','sea','s','d','gray','trade','kit','wilhelm','l','round','m',
                'statue','familiaris','art','soft','nike','para','marketplace','gift','wwf','womens','wildlife',
                'vinyl','lobo','dientes','clear','fishing','stretch','con','evolaboratory','natural','anatomy','du',
                'wedge','ebid','fisch','exotic','buddy','scentsy','spd','ocean','oz','tuvalu','pack','pcs','ee',
                'jxk','w','mit','t','hermes','mint','praparat','muscle','mount','postfrisch','original','skull',
                'taurus','bite','american','oil','fossilized','air','coin','bull','gator','da','squalo','bone',
                'home','der','collection','decoration','o','store','green','nuevo','grey','high','dress','gi',
                'free','sin','b','vtg','educational','joe','sticker','deko','fur','stamps','cd','hand','pesca',
                '1stdibs','neu','beach','gold','sz','design','seidenhai','philippinen','rezent','paperback',
                'malaysia','handmade','english','loup','lucchese','lithographie','regalo','feathers','figurine',
                'quill','hat','quality','dr','sleeve','y','diente','modelo','mini','resin','amber','wear',
                'collector','tee','golf','glacier']

In [11]:
common_words.extend(['timbre','micro','image','medal','medium','right','le','hombre','blanket','groe','zahn',
                    'holzstich','wedding','halloween','africano','costume','ball','porosus','embroidered',
                    'taschenbuch','zoll','pop','niger','pelliccia','components','shutterstock','funny','reef',
                    'gravure','ex','faune','meg','grande','decoracion','kelly','magazine','heart','scale','colgante',
                    'custom','sealed','hem','virginia','themed','nice','adult','patch','ltd','swim','adventures',
                    'th','david','power','sandtiger','pair','envueltos','day','in','hoodie','odo','signed','man',
                    'pelz','smooth','national','liberia','madera','fossiler','moon','glow','horror','brush','toro',
                    'tail','john','j','pez','water','italy','old','mountain','die','millinery','diving','impresion',
                    'graphite','hasbro','curse','pirotta','colours','crown','luxury','display','north','animales',
                    'los','wilderness','tecovas','nickelodeon','case','lp','tth','gran','excellent','collar',
                    'serie','accessories','purse','souvenir','cun','cafe','miniature','week','albania','ad','tag',
                    'king','nuovo','herren','complete','fine','zahne','bar','libro','colored','grade','informacion',
                    'movie','cotton','collecta','doll','offset','kid','pro','river','park','geschenk','sw','style',
                    'navy','bois','carte','kieran','crafts','ml','viking','tan','good','lote','ct','fleischwolf',
                    'south','animali','seuss','thailand','un','nwot','stampa','sealife','hanger','estampilla',
                    'talla','ring','common','anhanger','gk','shorts','strauss','models','saviour','w','tanzania',
                    'cap','figura','montado','co','coaster','ytfaulty','avec','module','tank','boot'])

In [12]:
common_words.extend(['you','especimen','gifts','paul','meucites','marvel','thriller','adjustable','tonyshells',
                     'whitetip','parts','orange','taxidermia','table','logo','base','greeting','satchel',
                     'plaschtier','select','su','beige','beautiful','sterling','chromo','rake','boy','little',
                     'jigging','is','deep','collectionneur','panatime','lemon','ultimate','lapel','sl','mouth','sa',
                     'sports','gem','camisa','fringse','islands','martin','nurse','pet','aquarium','inches','vida',
                     'star','hooks','sizes','gepard','sedoso','bahrein','sharp','tea','vba','aaa','near','pz',
                     'legno','plastic','estonie','toddler','kia','schwarz','clawdeen','vulgaris','creatures','ghw',
                     'pouces','lc','flounder','liver','crossbody','um','dautruche','pelt','shop','eesti','nbc',
                     'antenna','grado','casual','boilies','australian','ytpa','par','squali','collectible',
                     'mcdonalds','cross','cunicularia','dent','pearl','battles','bifold','anniversary','punk',
                     'maryland','moroccan','zman','brodtmann','rose','u','simulation','tribal','marble','grau',
                     'geldborse','postale','colori','strstampas','denti','interior','ornament','mn','figurines',
                     'bumper','completa','pool','clutch','aves','incisione','animale','forest','maldives','uganda',
                     'pinkfong','wooden','sobek','roper','echtes','baits','hoja','silvestre','euros','lps',
                     'baseball','eur','juego','fir','er','rangers','collezione','adventure','mom','age','lb',
                     'aspen','gordon','monk','mexico','jan','insecto','classics','lauren','shiny','clothing',
                     'prueba','cut','ak','roof','jody','paar','tiki','enhancement','harry','practical','tall',
                     'never','bambini','skirt','uomo','medalla','altos','range','predator','bobbers','sharky','vhs',
                     'boat','tiny','underwater','culture','mezlan','colors','gambie','my','cuna','hard','strong',
                     'about','congo','bf','interesting','scott','duster','daniels','enfants','veritable','huge',
                     'solid','schadel','proximus','tachys','dura','geographic','encyclopedia','champions','token',
                     'marshall','chili','halskette','etiquetas','nib','galapagos','davidson','alle','gloves',
                     'caspersen','calidad','anatomia','usado','pellet','dente','ns','buddies','main','jl','paint',
                     'colour','peces','sol','mosaic','ein','post','all','superb','open','tags','goodwin','part',
                     'arctos','organism','out','xx','circle','em','hobby','pajama','bond','pewter','bolsillo','deco',
                     'killer','halter','hornback','buy','cosplay','eye','colorful','wfbk','double','englisch',
                     'numbat','pat','flash','weird','sup'])

In [13]:
common_words.extend(['women','figure','picclick','ebay','material','print','jumper','tshirt','jewelry','belt','buckle','chain','watch',
                     'strap'])

In [14]:
len(common_words)

696

In [15]:
def remove_irrelevant_words(text):
    # Tokenize the text
    words = text.split()

    # Remove irrelevant words
    words = [word for word in words if word.lower() not in common_words]

    cleaned_text = ' '.join(words)

    return cleaned_text

In [16]:
df['product'] = df['product'].apply(remove_irrelevant_words)
df['product'].head(10)

0                 bones hailstones halibut mary oneill
1    contortus hill miocene physogaleus shark shark...
2           azaz contortus hill physogaleus sharktooth
3      contortus hill miocune physogaleus requin tigre
4    colline contortus miocune physogaleus requin t...
5    almofada brinquedo estereo gato peca pelucia s...
6    cuscino gatto giocattolo imbottito peluche sim...
7         decorations dining federn kanstliche ostrich
8                                  centerpiece ostrich
9                         assorted craft ostrich party
Name: product, dtype: object

## Species extraction using TaxoNERD
TaxoNERD is a domain-specific tool for recognizing taxon mentions in the biodiversity literature.
We will use the 'en_core_eco_biobert' model to extract species name from the pre-processed 'product_fingerprint' column.

In [19]:
taxonerd = TaxoNERD(prefer_gpu=False)

In [22]:
# Load TaxoNERD model
nlp = taxonerd.load(model="en_core_eco_biobert")

OSError: ignored

In [ ]:
# TaxoNERD example
taxonerd.find_in_text("band flat ostrich quick release")

In [ ]:
def get_species(name):
    try:
        doc = nlp(name)
        return doc.ents[0]
    except:
        return None

In [ ]:
# Count the number of nows that have unique 'name', 'product' and 'title' entries
unique_df = df[['name', 'product', 'title']].nunique()
unique_df

In [ ]:
# We will consider only unique product names to extract the species
unique_products = df['product'].unique()

In [ ]:
unique_products_df = pd.DataFrame({'unique_products': unique_products})
unique_products_df.head(10)

In [ ]:
unique_products_df['species'] = unique_products_df['unique_products'].apply(get_species)
unique_products_df.head(10)